In [12]:
import pandas as pd
import numpy as np
import math

# networkx
import networkx as nx

# transformers
from sentence_transformers import SentenceTransformer
import torch


# For semantic similarity
from urllib.parse import unquote
from sklearn.metrics.pairwise import cosine_similarity

# Python functions in .py file to read data
import data_readers

In [2]:
# The links and edges
wikispeedia = data_readers.read_wikispeedia_graph()

# The finished paths
finished_paths = data_readers.read_finished_paths()

# The unfinished paths
unfinished_paths = data_readers.read_unfinished_paths()

# DF of all articles
articles = data_readers.read_articles()

# DF of all articles and their categories
categories = data_readers.read_categories()

In [3]:
# We found out later that the data contained in the shortest path matrix given to us seems to be wrong
# Here we also add a quick dictionary that properly shows that this is wrong, and give an example
shortest_path_df = data_readers.read_shortest_path_df()
shortest_path_dict = dict(nx.all_pairs_shortest_path(wikispeedia))

# Searching for the string of a given article. It has to be formatted like the article name
# Which shouldn't be a problem, as we'll probably usually retrieve them internally
obi_wan_text = data_readers.plaintext_article_finder('Obi-Wan_Kenobi')

print("Dataset has", len(wikispeedia.nodes), "nodes (articles)")
print("Dataset has", len(wikispeedia.edges), "edges (links between articles)")

dic = nx.pagerank(wikispeedia)
print(dic)

for node in [nodo for nodo in wikispeedia.nodes()]:
    wikispeedia.nodes[node]['pagerank'] = dic[node]
print(wikispeedia.nodes(data=True))

Dataset has 4592 nodes (articles)
Dataset has 119882 edges (links between articles)
{'%C3%81ed%C3%A1n_mac_Gabr%C3%A1in': 3.2710390395592254e-05, 'Bede': 0.00021938161316650256, 'Columba': 0.00012116881897593145, 'D%C3%A1l_Riata': 0.000105560154678038, 'Great_Britain': 0.0015061292462843427, 'Ireland': 0.001899965414851622, 'Isle_of_Man': 0.00046032088563309313, 'Monarchy': 0.0008133221072287704, 'Orkney': 0.00028994588516574057, 'Picts': 0.00019759121101826146, 'Scotland': 0.0021425304277849183, 'Wales': 0.001099763207428743, '%C3%85land': 3.2710390395592254e-05, '20th_century': 0.0023615390664036124, 'Baltic_Sea': 0.0006497932466591777, 'Crimean_War': 0.0002201292664638352, 'Currency': 0.0032371559196565525, 'Euro': 0.0012495337101871686, 'European_Union': 0.002318582455038707, 'Finland': 0.001009621435681574, 'League_of_Nations': 0.0005769703027683626, 'List_of_countries_by_system_of_government': 0.0028333266533963103, 'Nationality': 0.00022617861790487603, 'Parliamentary_system': 0.

In [13]:
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to get embeddings using sentence transformer
def get_embedding(text):
    return model.encode(text, convert_to_tensor=True)

# Function to perform L2 normalization on the embeddings
def l2_normalize(tensor):
    return tensor / tensor.norm(p=2, dim=0, keepdim=True)

# Function to calculate semantic similarity between two pieces of text
def semantic_similarity(word1, word2):
    embedding1 = get_embedding(word1)
    embedding2 = get_embedding(word2)

    # L2 normalization of the embeddings (to make sure, although embedding should already be normalized)
    embedding1_normalized = l2_normalize(embedding1)
    embedding2_normalized = l2_normalize(embedding2)

    # Compute and return the similarity of normalized tensors
    return torch.dot(embedding1_normalized, embedding2_normalized).item()

In [49]:
semantic_similarity("Japan","Asia")

0.7521116733551025

In [170]:
# si funciona, podemos hacer sample para mirar resultados de ir buscando cambiando la ref_similarity

def degree_and_sem(G: nx.Graph, source: str, target: str, ref_similarity=0.3):
 # ref_similarity should be the avg sem dist

   visited = set([])
   current_children = []
   sem_sim_childr = {}
   max_page_childr = {}
   path = []

   current_node = source 

   found = False

   while(not found):
      visited.add(current_node)
      path.append(current_node)
      
      if len(path) >= 25:
         return path, "I give up, 50 or more visited nodes"

      if current_node == target:
        found = True
        return path, visited

      current_children = list(G.successors(current_node))
         
      sem_sim_childr = {}
      max_page_childr = {}
      # store in a dic each child and its sem sim
      for c in current_children:
         if c == target:
            found = True
            visited.add(c)
            path.append(c)
            return path, visited
         # compute semantic similarity
         elif c in visited:
            current_children.remove(c)
         else:
            semsim = semantic_similarity(c,target)
            #store it with ID and sem sim
            sem_sim_childr[c] = semsim
            # compute semantic similarity
            pagerank = G.nodes[c]['pagerank']
            #store it with ID and sem sim
            max_page_childr[c] = pagerank
    
      max_node = max(sem_sim_childr, key=sem_sim_childr.get)
      max_sim = sem_sim_childr[max_node]
      if max_sim >= ref_similarity:
         #se va al de mayor similarity
            current_node = max_node
      else:
         # se va al de mayor degree
         max_node = max(max_page_childr, key=max_page_childr.get)
         current_node = max_node

In [171]:
degree_and_sem(wikispeedia,"Quito","Water")

(['Quito', 'Basketball', 'Sport', 'Football', 'Sphere', 'Earth', 'Water'],
 {'Basketball', 'Earth', 'Football', 'Quito', 'Sphere', 'Sport', 'Water'})

In [172]:
degree_and_sem(wikispeedia,"Switzerland","Ant")

(['Switzerland',
  'Latin',
  'Latin_alphabet',
  'Alphabet',
  'Science',
  'Atom',
  'Rubidium',
  'Electron',
  'Amber',
  'Aristotle',
  'Moon',
  'Apollo_11',
  'Apollo_8',
  'Santa_Claus',
  'Toy',
  'Animal',
  'Insect',
  'Ant'],
 {'Alphabet',
  'Amber',
  'Animal',
  'Ant',
  'Apollo_11',
  'Apollo_8',
  'Aristotle',
  'Atom',
  'Electron',
  'Insect',
  'Latin',
  'Latin_alphabet',
  'Moon',
  'Rubidium',
  'Santa_Claus',
  'Science',
  'Switzerland',
  'Toy'})

In [173]:
degree_and_sem(wikispeedia,"Space_Shuttle_Columbia_disaster","Indus_Valley_Civilization")

(['Space_Shuttle_Columbia_disaster', 'India', 'Indus_Valley_Civilization'],
 {'India', 'Indus_Valley_Civilization', 'Space_Shuttle_Columbia_disaster'})

In [174]:
degree_and_sem(wikispeedia,"Western_Roman_Empire","Alcohol")

(['Western_Roman_Empire',
  'Cologne',
  'Bridge',
  'Steel',
  'Water',
  'Ethanol',
  'Alcohol'],
 {'Alcohol',
  'Bridge',
  'Cologne',
  'Ethanol',
  'Steel',
  'Water',
  'Western_Roman_Empire'})